In [ ]:
!pip install optuna

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy.random import rand
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import missingno as msno

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import optuna
from optuna.integration import TFKerasPruningCallback
from keras.backend import clear_session

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras import regularizers
from keras.utils.vis_utils import plot_model

from subprocess import check_output
import time
import warnings
warnings.filterwarnings('ignore')
import os

In [ ]:
def calcFamMembers(x, y):
    if x == 1:
        return y + 2
    else:
        return y + 2  
#x = Total income y= nr of family members 10% expense per person subtracted from total income    
def AvailIncome(x, y, z, n): 
    emi = float(z)*1000/float(n)
    return float(x) - (float(x) * 0.2 * float(y)) - emi

def Inc_Loan_Ratio(x,y):
    if x>0:
        return y*1000/x
    else:
        return 0
    
    
    
def makenumbers(combined, Training = False):    
    combined['Married'] = combined['Married'].replace({'No':0})
    combined['Married'] = combined['Married'].replace({'Yes':1})
    combined['Dependents'] = combined['Dependents'].replace({'0':0})
    combined['Dependents'] = combined['Dependents'].replace({'1':1})
    combined['Dependents'] = combined['Dependents'].replace({'2':2})
    combined['Dependents'] = combined['Dependents'].replace({'3+':3})    
    return combined


def multiplyloandata_pcversion(numberofmultiples, loandata, cols=[]):
    multiple = numberofmultiples
    rows = loandata.shape[0]
    print("input rows: ", rows)
    frames = []
    frames.append(loandata)
    for i in range(multiple):
        temp = loandata.copy(deep=True)
        for col in cols:
            temp[col] = (temp[col] + temp[col] * rand(rows) * 0.05).round(2)
        frames.append(temp)
    new = pd.concat(frames)
    new = new.sample(frac=1).reset_index(drop=True)
    print("output dataframe shape: ", new.shape)
    return new

In [ ]:
def processfeatures(combined, Training = False):
    
    if Training:
        combined['Credit_History'][combined['Loan_Status'] == 'Y'] = 1
        combined['Credit_History'][combined['Loan_Status'] == 'N'] = 0 
        combined['Loan_Status'] = combined['Loan_Status'].replace({'Y':1})
        combined['Loan_Status'] = combined['Loan_Status'].replace({'N':0})        
        combined.dropna(how='any', inplace = True)
    combined = combined.drop(['Loan_ID'], axis=1)
    #combined = combined.drop(['Married'], axis=1)
    


    combined['Gender'].fillna(combined['Gender'].mode()[0],inplace = True)
    combined['Married'].fillna(combined['Married'].mode()[0],inplace = True)
    combined['Education'].fillna(combined['Education'].mode()[0],inplace = True)
    combined['Dependents'].fillna(combined['Dependents'].mode()[0],inplace = True)
    combined['Loan_Amount_Term'].fillna(combined['Loan_Amount_Term'].mode()[0],inplace = True)
    combined['Loan_Amount_Term'] = combined['Loan_Amount_Term']
    combined['LoanAmount'].fillna(combined['LoanAmount'].mode()[0],inplace = True)
    combined['Self_Employed'].fillna(combined['Self_Employed'].mode()[0],inplace = True)    
    
    combined['Property_Area'].fillna(combined['Property_Area'].mode()[0],inplace = True)    
    combined['Total_income'] = combined['ApplicantIncome'] + combined['CoapplicantIncome']
    combined['Total_income'] = combined['Total_income']
    combined = makenumbers(combined, False)
    combined['AvailIncome'] = combined.apply(lambda 
                                             row: AvailIncome(row['Total_income'],row['Dependents'],row['LoanAmount'],row['Loan_Amount_Term']), axis=1)  
    combined['FMems'] = combined.apply(lambda row: calcFamMembers(row['Married'], row['Dependents']), axis=1)
    combined['Inc_Loan_Ratio'] = combined.apply(lambda row: Inc_Loan_Ratio(row['AvailIncome'], row['LoanAmount']), axis=1)
    addlcols = ['Married','Dependents']
    combined = combined.drop(addlcols, axis=1)
    categorise = ['Gender','Property_Area','Education','Self_Employed','Credit_History']
    combined = pd.get_dummies(combined, columns = categorise)
    return combined

In [ ]:
oritrain = pd.read_csv('../input/analytics-vidhya-loan-prediction/train.csv')
oritest = pd.read_csv('../input/analytics-vidhya-loan-prediction/test.csv')

In [ ]:
inputdf = processfeatures(oritrain, True)
scale_features = ['ApplicantIncome','CoapplicantIncome','AvailIncome','Total_income',
                  'LoanAmount','FMems','Loan_Amount_Term','Inc_Loan_Ratio']
#inputdf = multiplyloandata_pcversion(150, inputdf, cols=scale_features)
#inputdf[scale_features] = StandardScaler().fit_transform(inputdf[scale_features])
inputdf.head(2)

In [ ]:
dropcols = ['ApplicantIncome','CoapplicantIncome','Loan_Amount_Term','Total_income',
            'Gender_Male','Education_Graduate','Self_Employed_No','Credit_History_0.0']
            
inputdf = inputdf.drop(dropcols, axis=1)
inputdf.shape

In [ ]:
inputdf.head(2)

In [ ]:
sns.catplot(x="Loan_Status", y="AvailIncome", kind="swarm",hue="FMems", data=inputdf, height=5, aspect=2)
plt.show()

In [ ]:
sns.heatmap(
    data=inputdf.corr(),
    annot=True,
    fmt='.2f',
    cmap='RdYlGn'
)

fig = plt.gcf()
fig.set_size_inches(14, 10)

plt.show()

In [ ]:
inputdf.head(2)

In [ ]:
y = inputdf.Loan_Status
X = inputdf.drop(['Loan_Status'], axis = 1)

# Split your data into training and testing (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=42,
    test_size=0.30
)

In [ ]:
X_train.head(2)

In [ ]:
from tensorflow.python.keras.metrics import Metric
from tensorflow.python.keras.metrics import AUC
import functools
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import roc_auc_score
    
def as_keras_metric(method):

    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

def auroc(y_true, y_pred):
    return tf.py_function (roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
study_name = "LoanPredict"
checkpoint_path = './LoanPredict.hdf5'
checkpoint_dir = os.path.dirname(checkpoint_path)
saveBestmodel = keras.callbacks.ModelCheckpoint(checkpoint_path,
                              monitor='val_accuracy',
                              save_weights_only=False,
                              save_best_only=True,
                              verbose=0
                             )

def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    num_epochs = 600
        # Create callbacks for early stopping and pruning.
    callbacks = [
        keras.callbacks.EarlyStopping(patience=4),
        TFKerasPruningCallback(trial, "val_accuracy"),
        saveBestmodel
    ]    
    model = Sequential()
    for i in range(3):                        
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 16, 80, 8)),
                        activation = "relu",
                        input_dim=11
                       )
                 )              
    model.add(Dense(1, activation = 'sigmoid'))
    lr = trial.suggest_uniform("lr", 1e-5, 1e-3)
    model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=lr), metrics=['accuracy'])
    batch_size = trial.suggest_int('Batch_size', 256, 1024, 128) 
    history = model.fit(X_train, 
                       y_train,
                       validation_data= (X_test, y_test),
                       epochs=num_epochs,
                       batch_size = batch_size,
                       callbacks=callbacks,
                       verbose=0
                       )
    score = model.evaluate(X_test, y_test, verbose=1)
    return score[1]

In [ ]:
optuna.logging.disable_default_handler()
study = optuna.create_study(
        sampler=optuna.samplers.TPESampler(
            consider_prior=True, prior_weight=1.0, 
            consider_magic_clip=True, consider_endpoints=False, 
            n_startup_trials=10, n_ei_candidates=24, 
            seed=None), 
        pruner=optuna.pruners.SuccessiveHalvingPruner(
            min_resource=2, reduction_factor=4, min_early_stopping_rate=1),
        study_name = study_name, 
        direction="maximize",
)

#study.optimize(objective, n_trials=100, timeout=600)
study.optimize(objective, timeout=120)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial number:", study.best_trial.number)
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
#model = tf.keras.models.load_model(checkpoint_path)
#Load the optimized model which is saved at checkpoint_path
print(checkpoint_path)
new_model = keras.models.load_model(checkpoint_path)
new_model.summary()

In [ ]:
plot_model(new_model, show_shapes = True)

In [ ]:
preddf = processfeatures(oritest, False)
preddf[scale_features] = StandardScaler().fit_transform(preddf[scale_features])
preddf = preddf.drop(dropcols, axis=1)
y_predict = new_model.predict(preddf)
py_pred = np.where(y_predict > 0.45, 1, 0)

Resultdf= pd.Series(py_pred.ravel()).to_frame('Loan_Status')
Resultdf['Loan_Status'] = Resultdf['Loan_Status'].replace({0:'N'})
Resultdf['Loan_Status'] = Resultdf['Loan_Status'].replace({1:'Y'})
df = pd.DataFrame()
df['Loan_ID'] = oritest['Loan_ID']
df['Loan_Status'] = Resultdf['Loan_Status']
resfile = "./result3.csv"
df.to_csv(resfile, index=False, line_terminator='\n')